In [1]:
import math
import scipy as sp
import matplotlib.pyplot as plt
from scipy.stats import bernoulli, uniform, chi2
import numpy as np
from scipy.stats.mstats import gmean
from numpy.testing import assert_allclose
from utils import sprt_mart, alpha_mart, shrink_trunc, stratum_selector, multinomial_selector
np.random.seed(123456789)

In [2]:
theta = 1/2 
hand_tally = np.concatenate((np.ones(40), np.zeros(40), np.ones(60), np.zeros(40)))
reported_tally = np.concatenate((np.ones(40), np.zeros(40), np.ones(70), np.zeros(30)))
omega = reported_tally - hand_tally
strata = np.concatenate((np.ones(80), 2*np.ones(100)))


v = np.array([2 * np.mean(reported_tally[strata == 1]) - 1, 2 * np.mean(reported_tally[strata == 2]) - 1])
u = 2 / (2 - v)
stratum_1 = (1 - omega[strata == 1]) / (2 - v[0])
stratum_2 = (1 - omega[strata == 2]) / (2 - v[1])

In [3]:
shuffled_1 = np.random.permutation(stratum_1)
shuffled_2 = np.random.permutation(stratum_2)
N = np.concatenate((np.array([len(shuffled_1)]), np.array([len(shuffled_2)])))
w = N/sum(N)
epsilon = 1 / (2*np.max(N))
theta_1_grid = np.arange(epsilon, u[0] - epsilon, epsilon) #sequence from epsilon to u[0] - epsilon
theta_2_grid = (theta_1_grid - w[0] * theta_1_grid) / w[1]

In [14]:
#NEED TO CHECK THIS
#alpha_mart() seems to be returning negative numbers, which makes things fail downstreatm

x = shuffled_2
N = N[1]
mu = theta_2_grid[123]
eta = 1/(2-v[1])
estim = shrink_trunc
f = .01
u = u[1]
S = np.insert(np.cumsum(x),0,0)[0:-1]  # 0, x_1, x_1+x_2, ...,
j = np.arange(1,len(x)+1)              # 1, 2, 3, ..., len(x)
m = (N*mu-S)/(N-j+1) if np.isfinite(N) else mu   # mean of population after (j-1)st draw, if null is true
etaj = estim(x=x, N=N, mu=mu, nu=eta, f=f,u=u)
with np.errstate(divide='ignore',invalid='ignore'):
    terms = np.cumprod((x*etaj/m + (u-x)*(u-etaj)/(u-m))/u)
terms[m<0] = np.inf
terms[m>u] = 0
terms

array([1.00064004e+00, 1.00879109e+00, 1.01709307e+00, 1.02555039e+00,
       1.03416760e+00, 1.04294947e+00, 1.05190094e+00, 1.06102716e+00,
       1.07033349e+00, 1.07982553e+00, 1.08950910e+00, 1.09939029e+00,
       1.10947544e+00, 1.11977118e+00, 1.13028442e+00, 1.14102240e+00,
       1.15199268e+00, 1.16320318e+00, 1.17466218e+00, 1.18637834e+00,
       1.19836078e+00, 1.18622851e-04, 1.18600821e-04, 1.18578600e-04,
       1.18556182e-04, 1.18533560e-04, 1.18510727e-04, 1.18487678e-04,
       1.18464405e-04, 1.18440804e-04, 1.09892876e-04, 1.09759251e-04,
       1.09624338e-04, 1.09488095e-04, 1.09350473e-04, 1.09211427e-04,
       1.09070906e-04, 1.08928857e-04, 1.08785227e-04, 1.08639958e-04,
       1.08492991e-04, 1.08344264e-04, 1.08193710e-04, 1.00799962e-04,
       1.00535554e-04, 1.00267958e-04, 9.99970469e-05, 9.97226910e-05,
       9.94447514e-05, 9.91630817e-05, 9.88775274e-05, 9.85879251e-05,
       9.82941015e-05, 9.16246277e-05, 9.12117316e-05, 9.07928281e-05,
      

In [16]:
etaj

array([0.67      , 1.24993751, 1.24993751, 1.24993751, 1.24993751,
       1.24993751, 1.24993751, 1.24993751, 1.24993751, 1.24993751,
       1.24993751, 1.24993751, 1.24993751, 1.24993751, 1.24993751,
       1.24993751, 1.24993751, 1.24993751, 1.24993751, 1.24993751,
       1.24993751, 1.24993751, 0.67187044, 0.67170686, 0.67154606,
       0.67138803, 0.67123273, 0.67108015, 0.67093028, 0.670962  ,
       0.67176338, 0.67955477, 0.67954883, 0.67954953, 0.67955706,
       0.67957161, 0.6795934 , 0.67962264, 0.6796596 , 0.67970453,
       0.67975771, 0.67981946, 0.6798901 , 0.67997   , 0.69122024,
       0.69152274, 0.69184326, 0.6921827 , 0.69254205, 0.69292238,
       0.69332483, 0.69375065, 0.69420119, 0.69467792, 0.70876941,
       0.70960541, 0.71048658, 0.71141597, 0.71239691, 0.72867697,
       0.73015347, 0.74773885, 0.74981003, 0.75200086, 0.75432122,
       0.75678209, 0.75939576, 0.76217601, 0.76513834, 0.76830025,
       0.77168158, 0.77530492, 0.77919607, 0.78338467, 0.78790

In [7]:
selections = np.zeros((len(stratum_1) + len(stratum_2) - 1, len(theta_1_grid)))
intersection_marts = np.zeros((len(stratum_1) + len(stratum_2), len(theta_1_grid)))

for i in range(len(theta_1_grid)):
    mart_1 = alpha_mart(x = shuffled_1, N = N[0], mu = theta_1_grid[i], eta = 1/(2-v[0]), f = .01, u = u[0])
    mart_2 = alpha_mart(x = shuffled_2, N = N[1], mu = theta_2_grid[i], eta = 1/(2-v[1]), f = .01, u = u[1])
    intersection_marts[:,i] = stratum_selector(marts = [mart_1, mart_2], rule = multinomial_selector)[1]

/Users/Jake/Library/Python/3.8/lib/python/site-packages/scipy/stats/_stats_py.py:287: RuntimeWarning: divide by zero encountered in log
  log_a = np.log(a)
/Users/Jake/Dropbox/RLAs/alpha/Code/utils.py:217: RuntimeWarning: divide by zero encountered in true_divide
  ratios = running_T/geomean
/Users/Jake/Dropbox/RLAs/alpha/Code/utils.py:217: RuntimeWarning: invalid value encountered in true_divide
  ratios = running_T/geomean


ValueError: probabilities contain NaN

In [13]:
marts = [mart_1, mart_2]
marts[0]

array([9.87824675e-01, 8.80803936e-01, 7.84288484e-01, 6.97356505e-01,
       6.19157249e-01, 5.48906327e-01, 4.85881264e-01, 4.29417303e-01,
       3.78903449e-01, 3.33778728e-01, 2.93528672e-01, 2.57682006e-01,
       2.25807531e-01, 1.97511196e-01, 1.72433344e-01, 1.50246133e-01,
       1.30651117e-01, 1.13376974e-01, 9.81773905e-02, 8.48290745e-02,
       7.31299053e-02, 6.28972048e-02, 5.39661277e-02, 4.61881617e-02,
       3.94297343e-02, 3.35709170e-02, 2.85042250e-02, 2.41335038e-02,
       2.03728995e-02, 1.71459070e-02, 1.43844919e-02, 1.20282812e-02,
       1.00238182e-02, 8.32387893e-03, 6.88684404e-03, 5.67612427e-03,
       4.65963502e-03, 3.80931681e-03, 3.10069856e-03, 2.51250054e-03,
       2.02627409e-03, 1.62607551e-03, 1.29817138e-03, 1.03077305e-03,
       8.13797783e-04, 6.38654682e-04, 4.98053060e-04, 3.85831578e-04,
       2.96806240e-04, 2.26635619e-04, 1.71701730e-04, 1.29005092e-04,
       9.60726333e-05, 7.08771639e-05, 5.17672506e-05, 3.74064143e-05,
      